

### Why this structure
- The src layout isolates importable code from the repository root, preventing accidental imports and improving packaging hygiene during development and testing.[6][1]
- The argparse subparser pattern provides a discoverable CLI with grouped commands for users, tickets, and reports without external dependencies.[5]
- SQLite with PRAGMA foreign_keys enforces relationships between users, tickets, and comments while keeping setup zero-config and file-based for easy onboarding.[4][7]

### Project tree
- This layout separates domain models, services, CLI, utilities, tests, and data, which aligns with common guidance on structuring maintainable Python projects.[2][3]

```
support_ticket_cli/
│
├── src/
│   ├── __init__.py
│   ├── main.py
│   ├── models/
│   │   ├── __init__.py
│   │   ├── database.py
│   │   ├── user.py
│   │   ├── ticket.py
│   │   └── comment.py
│   ├── utils/
│   │   ├── __init__.py
│   │   ├── validators.py
│   │   ├── helpers.py
│   │   └── logger.py
│   ├── services/
│   │   ├── __init__.py
│   │   ├── user_service.py
│   │   ├── ticket_service.py
│   │   └── report_service.py
│   └── cli/
│       ├── __init__.py
│       ├── menu.py
│       └── interface.py
│
├── data/
│   └── tickets.db
│
├── tests/
│   ├── __init__.py
│   ├── test_utils.py
│   ├── test_services.py
│   └── test_models.py
│
├── requirements.txt
├── README.md
└── .gitignore
```

### Setup and run
- Use a virtual environment, install requirements, then run the interactive menu with no args or subcommands via argparse for automation.[5]
- The database file is created on first connection, and foreign key enforcement is enabled per connection using PRAGMA foreign_keys, which is required for SQLite to enforce referential integrity.[7][4]

### src/__init__.py
- Package initializer for discoverability and a simple version constant.[3]

```python
# src/__init__.py
__all__ = ["models", "services", "utils", "cli"]
__version__ = "0.1.0"
```

### src/main.py
- Entry point selects interactive menu if no arguments are provided or runs the CLI when subcommands are passed.[5]

```python
# src/main.py
from src.cli.menu import run_menu
from src.cli.interface import cli_main
import sys

def main() -> None:
    if len(sys.argv) == 1:
        run_menu()
    else:
        cli_main()

if __name__ == "__main__":
    main()
```

### src/models/database.py
- Centralized connection management enables Row access, enforces foreign keys, and initializes a schema for users, tickets, and comments.[4]

```python
# src/models/database.py
from __future__ import annotations

import sqlite3
from pathlib import Path
from contextlib import contextmanager
from typing import Iterator, Optional
from src.utils.logger import get_logger

LOGGER = get_logger(__name__)

class Database:
    def __init__(self, db_path: Optional[Path] = None) -> None:
        if db_path is None:
            project_root = Path(__file__).resolve().parent.parent.parent
            db_path = project_root / "data" / "tickets.db"
        self.db_path = Path(db_path)
        self.db_path.parent.mkdir(parents=True, exist_ok=True)

    def connect(self) -> sqlite3.Connection:
        conn = sqlite3.connect(
            self.db_path,
            timeout=10.0,
            isolation_level=None,
            detect_types=sqlite3.PARSE_DECLTYPES | sqlite3.PARSE_COLNAMES,
            check_same_thread=False,
        )
        conn.row_factory = sqlite3.Row
        conn.execute("PRAGMA foreign_keys = ON;")
        return conn

    @contextmanager
    def get_connection(self) -> Iterator[sqlite3.Connection]:
        conn = self.connect()
        try:
            yield conn
        except Exception:
            LOGGER.exception("Database error, rolling back")
            conn.rollback()
            raise
        finally:
            conn.close()

    def initialize_schema(self) -> None:
        schema = """
        BEGIN;

        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            email TEXT UNIQUE NOT NULL,
            created_at TEXT NOT NULL
        );

        CREATE TABLE IF NOT EXISTS tickets (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT NOT NULL,
            description TEXT NOT NULL,
            status TEXT NOT NULL CHECK(status IN ('OPEN','IN_PROGRESS','RESOLVED','CLOSED')),
            priority TEXT NOT NULL CHECK(priority IN ('LOW','MEDIUM','HIGH','URGENT')),
            requester_id INTEGER NOT NULL,
            assignee_id INTEGER,
            created_at TEXT NOT NULL,
            updated_at TEXT NOT NULL,
            resolved_at TEXT,
            FOREIGN KEY(requester_id) REFERENCES users(id) ON DELETE RESTRICT,
            FOREIGN KEY(assignee_id) REFERENCES users(id) ON DELETE SET NULL
        );

        CREATE TABLE IF NOT EXISTS comments (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            ticket_id INTEGER NOT NULL,
            author_id INTEGER NOT NULL,
            body TEXT NOT NULL,
            created_at TEXT NOT NULL,
            FOREIGN KEY(ticket_id) REFERENCES tickets(id) ON DELETE CASCADE,
            FOREIGN KEY(author_id) REFERENCES users(id) ON DELETE RESTRICT
        );

        CREATE INDEX IF NOT EXISTS idx_tickets_status ON tickets(status);
        CREATE INDEX IF NOT EXISTS idx_tickets_assignee ON tickets(assignee_id);
        COMMIT;
        """
        with self.get_connection() as conn:
            conn.executescript(schema)
```

### src/models/user.py
- Lightweight dataclass for users to type responses and convert from sqlite rows.[3]

```python
# src/models/user.py
from __future__ import annotations
from dataclasses import dataclass
from typing import Optional
import sqlite3

@dataclass(slots=True)
class User:
    id: Optional[int]
    name: str
    email: str
    created_at: str

    @classmethod
    def from_row(cls, row: sqlite3.Row) -> "User":
        return cls(
            id=row["id"],
            name=row["name"],
            email=row["email"],
            created_at=row["created_at"],
        )
```

### src/models/ticket.py
- Ticket dataclass for core issue tracking properties like status and priority.[3]

```python
# src/models/ticket.py
from __future__ import annotations
from dataclasses import dataclass
from typing import Optional
import sqlite3

@dataclass(slots=True)
class Ticket:
    id: Optional[int]
    title: str
    description: str
    status: str
    priority: str
    requester_id: int
    assignee_id: Optional[int]
    created_at: str
    updated_at: str
    resolved_at: Optional[str]

    @classmethod
    def from_row(cls, row: sqlite3.Row) -> "Ticket":
        return cls(
            id=row["id"],
            title=row["title"],
            description=row["description"],
            status=row["status"],
            priority=row["priority"],
            requester_id=row["requester_id"],
            assignee_id=row["assignee_id"],
            created_at=row["created_at"],
            updated_at=row["updated_at"],
            resolved_at=row["resolved_at"],
        )
```

### src/models/comment.py
- Comment dataclass linking users and tickets to messages for audit trails.[3]

```python
# src/models/comment.py
from __future__ import annotations
from dataclasses import dataclass
from typing import Optional
import sqlite3

@dataclass(slots=True)
class Comment:
    id: Optional[int]
    ticket_id: int
    author_id: int
    body: str
    created_at: str

    @classmethod
    def from_row(cls, row: sqlite3.Row) -> "Comment":
        return cls(
            id=row["id"],
            ticket_id=row["ticket_id"],
            author_id=row["author_id"],
            body=row["body"],
            created_at=row["created_at"],
        )
```

### src/utils/validators.py
- Validators for email, title, status, priority, and text fields keep inputs clean before persistence.[3]

```python
# src/utils/validators.py
from __future__ import annotations
import re

EMAIL_RE = re.compile(r"^[^@\s]+@[^@\s]+\.[^@\s]+$")

VALID_STATUSES = {"OPEN", "IN_PROGRESS", "RESOLVED", "CLOSED"}
VALID_PRIORITIES = {"LOW", "MEDIUM", "HIGH", "URGENT"}

def validate_email(email: str) -> str:
    email = email.strip()
    if not EMAIL_RE.match(email):
        raise ValueError("Invalid email format")
    return email

def validate_nonempty(text: str, field: str, max_len: int = 255) -> str:
    value = text.strip()
    if not value:
        raise ValueError(f"{field} must not be empty")
    if len(value) > max_len:
        raise ValueError(f"{field} too long (>{max_len})")
    return value

def validate_status(status: str) -> str:
    s = status.strip().upper()
    if s not in VALID_STATUSES:
        raise ValueError(f"Status must be one of {sorted(VALID_STATUSES)}")
    return s

def validate_priority(priority: str) -> str:
    p = priority.strip().upper()
    if p not in VALID_PRIORITIES:
        raise ValueError(f"Priority must be one of {sorted(VALID_PRIORITIES)}")
    return p

def validate_id(value: int, field: str = "id") -> int:
    try:
        v = int(value)
    except Exception as e:
        raise ValueError(f"{field} must be integer") from e
    if v <= 0:
        raise ValueError(f"{field} must be positive")
    return v
```

### src/utils/helpers.py
- Helpers for timestamps and simple formatting centralize consistent behavior across modules.[3]

```python
# src/utils/helpers.py
from __future__ import annotations
from datetime import datetime

def now_iso() -> str:
    return datetime.utcnow().isoformat(timespec="seconds") + "Z"
```

### src/utils/logger.py
- Minimal logging setup to keep diagnostics uniform without external libraries.[3]

```python
# src/utils/logger.py
from __future__ import annotations
import logging
from logging import Logger

def get_logger(name: str) -> Logger:
    logger = logging.getLogger(name)
    if not logger.handlers:
        logger.setLevel(logging.INFO)
        ch = logging.StreamHandler()
        ch.setLevel(logging.INFO)
        fmt = logging.Formatter(
            "%(asctime)s | %(levelname)s | %(name)s | %(message)s",
            datefmt="%Y-%m-%d %H:%M:%S",
        )
        ch.setFormatter(fmt)
        logger.addHandler(ch)
        logger.propagate = False
    return logger
```

### src/services/user_service.py
- User service manages creation and listing of users, validating emails and names before writes.[3]

```python
# src/services/user_service.py
from __future__ import annotations
from typing import List
from src.models.database import Database
from src.models.user import User
from src.utils.validators import validate_email, validate_nonempty
from src.utils.helpers import now_iso

class UserService:
    def __init__(self, db: Database) -> None:
        self.db = db

    def add_user(self, name: str, email: str) -> int:
        name = validate_nonempty(name, "name", max_len=100)
        email = validate_email(email)
        ts = now_iso()
        with self.db.get_connection() as conn:
            cur = conn.cursor()
            cur.execute("BEGIN;")
            cur.execute(
                "INSERT INTO users (name, email, created_at) VALUES (?, ?, ?)",
                (name, email, ts),
            )
            uid = int(cur.lastrowid)
            cur.execute("COMMIT;")
            return uid

    def list_users(self) -> List[User]:
        with self.db.get_connection() as conn:
            rows = conn.execute("SELECT * FROM users ORDER BY name ASC").fetchall()
            return [User.from_row(r) for r in rows]
```

### src/services/ticket_service.py
- Ticket service encapsulates ticket CRUD, assignment, status changes, and comments using transactional operations with foreign keys enforced.[4]

```python
# src/services/ticket_service.py
from __future__ import annotations
from typing import List, Optional, Tuple
from src.models.database import Database
from src.models.ticket import Ticket
from src.models.comment import Comment
from src.utils.validators import (
    validate_nonempty,
    validate_status,
    validate_priority,
    validate_id,
)
from src.utils.helpers import now_iso

class TicketService:
    def __init__(self, db: Database) -> None:
        self.db = db

    def create_ticket(
        self,
        title: str,
        description: str,
        requester_id: int,
        priority: str = "MEDIUM",
    ) -> int:
        title = validate_nonempty(title, "title", max_len=200)
        description = validate_nonempty(description, "description", max_len=4000)
        requester_id = validate_id(requester_id, "requester_id")
        priority = validate_priority(priority)
        ts = now_iso()
        with self.db.get_connection() as conn:
            cur = conn.cursor()
            cur.execute("BEGIN;")
            cur.execute(
                """
                INSERT INTO tickets
                (title, description, status, priority, requester_id, assignee_id, created_at, updated_at, resolved_at)
                VALUES (?, ?, 'OPEN', ?, ?, NULL, ?, ?, NULL)
                """,
                (title, description, priority, requester_id, ts, ts),
            )
            tid = int(cur.lastrowid)
            cur.execute("COMMIT;")
            return tid

    def update_ticket(
        self,
        ticket_id: int,
        title: Optional[str] = None,
        description: Optional[str] = None,
        priority: Optional[str] = None,
    ) -> None:
        ticket_id = validate_id(ticket_id, "ticket_id")
        fields = []
        params = []
        if title is not None:
            fields.append("title = ?")
            params.append(validate_nonempty(title, "title", 200))
        if description is not None:
            fields.append("description = ?")
            params.append(validate_nonempty(description, "description", 4000))
        if priority is not None:
            fields.append("priority = ?")
            params.append(validate_priority(priority))
        if not fields:
            return
        params.extend([now_iso(), ticket_id])
        q = f"UPDATE tickets SET {', '.join(fields)}, updated_at = ? WHERE id = ?"
        with self.db.get_connection() as conn:
            cur = conn.cursor()
            cur.execute("BEGIN;")
            cur.execute(q, params)
            if cur.rowcount == 0:
                cur.execute("ROLLBACK;")
                raise LookupError("Ticket not found")
            cur.execute("COMMIT;")

    def assign_ticket(self, ticket_id: int, assignee_id: Optional[int]) -> None:
        ticket_id = validate_id(ticket_id, "ticket_id")
        if assignee_id is not None:
            assignee_id = validate_id(assignee_id, "assignee_id")
        with self.db.get_connection() as conn:
            cur = conn.cursor()
            cur.execute("BEGIN;")
            # verify ticket exists
            t = cur.execute("SELECT id FROM tickets WHERE id = ?", (ticket_id,)).fetchone()
            if not t:
                cur.execute("ROLLBACK;")
                raise LookupError("Ticket not found")
            # verify user exists if provided
            if assignee_id is not None:
                u = cur.execute("SELECT id FROM users WHERE id = ?", (assignee_id,)).fetchone()
                if not u:
                    cur.execute("ROLLBACK;")
                    raise LookupError("Assignee not found")
            cur.execute(
                "UPDATE tickets SET assignee_id = ?, updated_at = ? WHERE id = ?",
                (assignee_id, now_iso(), ticket_id),
            )
            cur.execute("COMMIT;")

    def change_status(self, ticket_id: int, status: str) -> None:
        ticket_id = validate_id(ticket_id, "ticket_id")
        status = validate_status(status)
        ts = now_iso()
        resolved_at = ts if status in {"RESOLVED", "CLOSED"} else None
        with self.db.get_connection() as conn:
            cur = conn.cursor()
            cur.execute("BEGIN;")
            cur.execute(
                "UPDATE tickets SET status = ?, updated_at = ?, resolved_at = ? WHERE id = ?",
                (status, ts, resolved_at, ticket_id),
            )
            if cur.rowcount == 0:
                cur.execute("ROLLBACK;")
                raise LookupError("Ticket not found")
            cur.execute("COMMIT;")

    def add_comment(self, ticket_id: int, author_id: int, body: str) -> int:
        ticket_id = validate_id(ticket_id, "ticket_id")
        author_id = validate_id(author_id, "author_id")
        body = validate_nonempty(body, "comment", max_len=4000)
        ts = now_iso()
        with self.db.get_connection() as conn:
            cur = conn.cursor()
            cur.execute("BEGIN;")
            # ensure related rows exist
            t = cur.execute("SELECT id FROM tickets WHERE id = ?", (ticket_id,)).fetchone()
            u = cur.execute("SELECT id FROM users WHERE id = ?", (author_id,)).fetchone()
            if not t or not u:
                cur.execute("ROLLBACK;")
                raise LookupError("Ticket or author not found")
            cur.execute(
                "INSERT INTO comments (ticket_id, author_id, body, created_at) VALUES (?, ?, ?, ?)",
                (ticket_id, author_id, body, ts),
            )
            cid = int(cur.lastrowid)
            cur.execute("COMMIT;")
            return cid

    def get_ticket(self, ticket_id: int) -> Ticket:
        ticket_id = validate_id(ticket_id, "ticket_id")
        with self.db.get_connection() as conn:
            row = conn.execute("SELECT * FROM tickets WHERE id = ?", (ticket_id,)).fetchone()
            if not row:
                raise LookupError("Ticket not found")
            return Ticket.from_row(row)

    def list_tickets(
        self,
        status: Optional[str] = None,
        assignee_id: Optional[int] = None,
    ) -> List[Ticket]:
        q = "SELECT * FROM tickets"
        clauses = []
        params: list = []
        if status:
            clauses.append("status = ?")
            params.append(validate_status(status))
        if assignee_id:
            clauses.append("assignee_id = ?")
            params.append(validate_id(assignee_id, "assignee_id"))
        if clauses:
            q += " WHERE " + " AND ".join(clauses)
        q += " ORDER BY updated_at DESC"
        with self.db.get_connection() as conn:
            rows = conn.execute(q, params).fetchall()
            return [Ticket.from_row(r) for r in rows]

    def list_comments(self, ticket_id: int) -> List[Comment]:
        ticket_id = validate_id(ticket_id, "ticket_id")
        with self.db.get_connection() as conn:
            rows = conn.execute(
                "SELECT * FROM comments WHERE ticket_id = ? ORDER BY created_at ASC",
                (ticket_id,),
            ).fetchall()
            return [Comment.from_row(r) for r in rows]
```

### src/services/report_service.py
- Reporting service provides status counts and average resolution time using SQL aggregation, suitable for hands-on SQL practice.[3]

```python
# src/services/report_service.py
from __future__ import annotations
from typing import Dict, Optional
from src.models.database import Database

class ReportService:
    def __init__(self, db: Database) -> None:
        self.db = db

    def status_counts(self) -> Dict[str, int]:
        with self.db.get_connection() as conn:
            rows = conn.execute(
                "SELECT status, COUNT(*) AS c FROM tickets GROUP BY status"
            ).fetchall()
            return {r["status"]: int(r["c"]) for r in rows}

    def average_resolution_hours(self) -> Optional[float]:
        # Average hours from created_at to resolved_at for resolved/closed tickets
        with self.db.get_connection() as conn:
            row = conn.execute(
                """
                SELECT AVG(
                    (julianday(resolved_at) - julianday(created_at)) * 24.0
                ) AS avg_hours
                FROM tickets
                WHERE resolved_at IS NOT NULL
                """
            ).fetchone()
            return float(row["avg_hours"]) if row and row["avg_hours"] is not None else None
```

### src/cli/menu.py
- Interactive menu supports learning flow while sharing the same services as the CLI for consistency.[3]

```python
# src/cli/menu.py
from __future__ import annotations
from src.models.database import Database
from src.services.user_service import UserService
from src.services.ticket_service import TicketService
from src.services.report_service import ReportService

def run_menu() -> None:
    db = Database()
    db.initialize_schema()
    users = UserService(db)
    tickets = TicketService(db)
    reports = ReportService(db)

    while True:
        print("\nSupport Ticketing CLI")
        print("1) Add user")
        print("2) List users")
        print("3) Create ticket")
        print("4) Update ticket")
        print("5) Assign ticket")
        print("6) Change status")
        print("7) Add comment")
        print("8) View ticket")
        print("9) List tickets")
        print("10) Status counts")
        print("11) Avg resolution hours")
        print("0) Exit")

        choice = input("Select: ").strip()
        try:
            if choice == "1":
                name = input("Name: ")
                email = input("Email: ")
                uid = users.add_user(name, email)
                print(f"User added id={uid}")
            elif choice == "2":
                for u in users.list_users():
                    print(f"{u.id}\t{u.name}\t{u.email}")
            elif choice == "3":
                title = input("Title: ")
                desc = input("Description: ")
                requester_id = int(input("Requester ID: "))
                priority = input("Priority [LOW|MEDIUM|HIGH|URGENT] (default MEDIUM): ") or "MEDIUM"
                tid = tickets.create_ticket(title, desc, requester_id, priority)
                print(f"Ticket created id={tid}")
            elif choice == "4":
                tid = int(input("Ticket ID: "))
                title = input("New title (blank skip): ") or None
                desc = input("New description (blank skip): ") or None
                priority = input("New priority (blank skip): ") or None
                tickets.update_ticket(tid, title=title, description=desc, priority=priority)
                print("Updated.")
            elif choice == "5":
                tid = int(input("Ticket ID: "))
                raw = input("Assignee ID (blank to unassign): ")
                assignee_id = int(raw) if raw.strip() else None
                tickets.assign_ticket(tid, assignee_id)
                print("Assigned.")
            elif choice == "6":
                tid = int(input("Ticket ID: "))
                status = input("Status [OPEN|IN_PROGRESS|RESOLVED|CLOSED]: ")
                tickets.change_status(tid, status)
                print("Status changed.")
            elif choice == "7":
                tid = int(input("Ticket ID: "))
                author = int(input("Author ID: "))
                body = input("Comment: ")
                cid = tickets.add_comment(tid, author, body)
                print(f"Comment added id={cid}")
            elif choice == "8":
                tid = int(input("Ticket ID: "))
                t = tickets.get_ticket(tid)
                print(f"[{t.id}] {t.title} | {t.status} | {t.priority} | req={t.requester_id} | asg={t.assignee_id}")
                for c in tickets.list_comments(tid):
                    print(f"  - ({c.created_at}) {c.author_id}: {c.body}")
            elif choice == "9":
                st = input("Filter status (blank for any): ") or None
                asg = input("Filter assignee id (blank for any): ")
                asg_id = int(asg) if asg.strip() else None
                for t in tickets.list_tickets(status=st, assignee_id=asg_id):
                    print(f"[{t.id}] {t.title} [{t.status}] <{t.priority}>")
            elif choice == "10":
                counts = reports.status_counts()
                for k, v in counts.items():
                    print(f"{k}: {v}")
            elif choice == "11":
                avg = reports.average_resolution_hours()
                print(f"Average resolution hours: {avg if avg is not None else 'N/A'}")
            elif choice == "0":
                print("Bye.")
                return
            else:
                print("Invalid selection.")
        except Exception as e:
            print(f"Error: {e}")
```

### src/cli/interface.py
- Argparse CLI with subcommands mirrors the menu and illustrates best practice with subparsers for clean help and grouping.[5]

```python
# src/cli/interface.py
from __future__ import annotations
import argparse
from src.models.database import Database
from src.services.user_service import UserService
from src.services.ticket_service import TicketService
from src.services.report_service import ReportService

def cli_main(argv: list[str] | None = None) -> None:
    parser = argparse.ArgumentParser(prog="support", description="Customer Support Ticketing CLI")
    sub = parser.add_subparsers(dest="cmd", required=True)

    # Users
    u_add = sub.add_parser("users:add", help="Add a user")
    u_add.add_argument("--name", required=True)
    u_add.add_argument("--email", required=True)

    u_list = sub.add_parser("users:list", help="List users")

    # Tickets
    t_create = sub.add_parser("tickets:create", help="Create a ticket")
    t_create.add_argument("--title", required=True)
    t_create.add_argument("--description", required=True)
    t_create.add_argument("--requester-id", type=int, required=True)
    t_create.add_argument("--priority", choices=["LOW","MEDIUM","HIGH","URGENT"], default="MEDIUM")

    t_update = sub.add_parser("tickets:update", help="Update a ticket")
    t_update.add_argument("--id", type=int, required=True)
    t_update.add_argument("--title")
    t_update.add_argument("--description")
    t_update.add_argument("--priority", choices=["LOW","MEDIUM","HIGH","URGENT"])

    t_assign = sub.add_parser("tickets:assign", help="Assign/unassign a ticket")
    t_assign.add_argument("--id", type=int, required=True)
    t_assign.add_argument("--assignee-id", type=int)

    t_status = sub.add_parser("tickets:status", help="Change status")
    t_status.add_argument("--id", type=int, required=True)
    t_status.add_argument("--status", choices=["OPEN","IN_PROGRESS","RESOLVED","CLOSED"], required=True)

    t_comment = sub.add_parser("tickets:comment", help="Add comment")
    t_comment.add_argument("--ticket-id", type=int, required=True)
    t_comment.add_argument("--author-id", type=int, required=True)
    t_comment.add_argument("--body", required=True)

    t_view = sub.add_parser("tickets:view", help="View a ticket")
    t_view.add_argument("--id", type=int, required=True)

    t_list = sub.add_parser("tickets:list", help="List tickets")
    t_list.add_argument("--status", choices=["OPEN","IN_PROGRESS","RESOLVED","CLOSED"])
    t_list.add_argument("--assignee-id", type=int)

    # Reports
    r_counts = sub.add_parser("reports:status-counts", help="Status bucket counts")
    r_avg = sub.add_parser("reports:avg-resolution", help="Average resolution time (hours)")

    args = parser.parse_args(argv)

    db = Database()
    db.initialize_schema()
    users = UserService(db)
    tickets = TicketService(db)
    reports = ReportService(db)

    if args.cmd == "users:add":
        uid = users.add_user(args.name, args.email)
        print(uid)
    elif args.cmd == "users:list":
        for u in users.list_users():
            print(f"{u.id}\t{u.name}\t{u.email}")
    elif args.cmd == "tickets:create":
        tid = tickets.create_ticket(args.title, args.description, args.requester_id, args.priority)
        print(tid)
    elif args.cmd == "tickets:update":
        tickets.update_ticket(args.id, title=args.title, description=args.description, priority=args.priority)
        print("OK")
    elif args.cmd == "tickets:assign":
        tickets.assign_ticket(args.id, args.assignee_id if "assignee_id" in args else None)
        print("OK")
    elif args.cmd == "tickets:status":
        tickets.change_status(args.id, args.status)
        print("OK")
    elif args.cmd == "tickets:comment":
        cid = tickets.add_comment(args.ticket_id, args.author_id, args.body)
        print(cid)
    elif args.cmd == "tickets:view":
        t = tickets.get_ticket(args.id)
        print(f"[{t.id}] {t.title} | {t.status} | {t.priority} | req={t.requester_id} | asg={t.assignee_id}")
        for c in tickets.list_comments(args.id):
            print(f"  - ({c.created_at}) {c.author_id}: {c.body}")
    elif args.cmd == "tickets:list":
        for t in tickets.list_tickets(status=args.status, assignee_id=args.assignee_id):
            print(f"[{t.id}] {t.title} [{t.status}] <{t.priority}>")
    elif args.cmd == "reports:status-counts":
        print(reports.status_counts())
    elif args.cmd == "reports:avg-resolution":
        print(reports.average_resolution_hours())
```

### tests/test_utils.py
- Utility tests validate the most important boundary checks for consistent input handling.[3]

```python
# tests/test_utils.py
import pytest
from src.utils.validators import (
    validate_email, validate_nonempty, validate_status, validate_priority, validate_id
)

def test_email_ok():
    assert validate_email("a@b.com") == "a@b.com"

def test_nonempty_ok():
    assert validate_nonempty(" x ", "x") == "x"

def test_status_ok():
    assert validate_status("open") == "OPEN"

def test_priority_ok():
    assert validate_priority("high") == "HIGH"

def test_id_positive():
    assert validate_id(5) == 5

def test_id_invalid():
    with pytest.raises(ValueError):
        validate_id(0)
```

### tests/test_services.py
- Service tests verify user creation, ticket lifecycle, and comment insertion using an isolated SQLite file.[4]

```python
# tests/test_services.py
from pathlib import Path
from src.models.database import Database
from src.services.user_service import UserService
from src.services.ticket_service import TicketService
from src.services.report_service import ReportService

def test_ticket_lifecycle(tmp_path: Path):
    db = Database(db_path=tmp_path / "test.db")
    db.initialize_schema()
    users = UserService(db)
    tickets = TicketService(db)
    reports = ReportService(db)

    req_id = users.add_user("Requester", "req@example.com")
    asg_id = users.add_user("Agent", "agent@example.com")

    tid = tickets.create_ticket("Cannot login", "Login fails with 500", req_id, "HIGH")
    tickets.assign_ticket(tid, asg_id)
    tickets.add_comment(tid, req_id, "Happens since yesterday")
    tickets.change_status(tid, "IN_PROGRESS")
    tickets.change_status(tid, "RESOLVED")

    t = tickets.get_ticket(tid)
    assert t.status == "RESOLVED"
    counts = reports.status_counts()
    assert counts["RESOLVED"] >= 1
```

### tests/test_models.py
- Simple dataclass sanity checks help catch unintended changes to row-to-model conversions.[3]

```python
# tests/test_models.py
from src.models.ticket import Ticket

def test_ticket_model_fields():
    t = Ticket(
        id=None,
        title="T",
        description="D",
        status="OPEN",
        priority="LOW",
        requester_id=1,
        assignee_id=None,
        created_at="2025-01-01T00:00:00Z",
        updated_at="2025-01-01T00:00:00Z",
        resolved_at=None,
    )
    assert t.status == "OPEN"
    assert t.priority == "LOW"
```

### requirements.txt
- Only pytest is required for tests; the application uses standard library modules for CLI and database access.[5]

```
pytest>=7.0
```

### README.md
- The README documents setup, commands, and how the project maps to Module 1 topics, reinforcing src layout, argparse CLI, and SQLite usage.[1][5]

```markdown
# Customer Support Ticketing CLI

A teaching-friendly, production-quality CLI for customer support tickets using a src layout, SQLite, and argparse.

## Quickstart
1) Create venv
   python -m venv .venv && . .venv/bin/activate  (Windows: .venv\Scripts\activate)

2) Install
   pip install -r requirements.txt

3) Run
   python -m src.main
   # or CLI subcommands:
   python -m src.main users:add --name "Alice" --email alice@example.com
   python -m src.main tickets:create --title "Login issue" --description "500 error" --requester-id 1 --priority HIGH
   python -m src.main tickets:list --status OPEN
   python -m src.main reports:status-counts

## Commands
- users:add --name --email
- users:list
- tickets:create --title --description --requester-id [--priority]
- tickets:update --id [--title] [--description] [--priority]
- tickets:assign --id [--assignee-id]
- tickets:status --id --status
- tickets:comment --ticket-id --author-id --body
- tickets:view --id
- tickets:list [--status] [--assignee-id]
- reports:status-counts
- reports:avg-resolution

## How this maps to Module 1
- Core Python and data types: services, CLI, and validators use numbers, strings, booleans, and casting.
- Operators and control flow: conditionals and loops appear throughout services and CLI handling.
- Functions and modules: reusable functions and a layered module structure.
- OOP: dataclasses for User, Ticket, Comment represent domain entities.
- Standard library: argparse for CLI, sqlite3 for DB, logging for diagnostics.
- File I/O and SQL: SQLite database file, DDL/DML, and SQL queries with grouping and joins (via foreign keys).

## Tests
Run:
   pytest -q
```

### .gitignore
- Ignores Python caches, virtual environments, and local databases to keep the repo clean and reproducible.[3]

```
# Python
__pycache__/
*.pyc
*.pyo
*.pyd
*.pytest_cache/
*.coverage
.coverage*
htmlcov/

# Envs
.venv/
env/
venv/

# OS / Editors
.DS_Store
.idea/
.vscode/
*.swp

# Local DBs
*.sqlite
*.db
!.gitkeep
```

### Module 1 coverage
- The project exercises syntax, control flow, data structures, functions, OOP, the standard library, file I/O, SQL DDL/DML, and Python-to-SQL integration through realistic ticketing workflows and reports, enabling a week-by-week progression aligned to the requested curriculum.[5][3]
- Using src layout, argparse subparsers, and SQLite with foreign keys ties the fundamentals to professional practices suitable for both learning and production-style organization.[1][4][5]

### Notes on SQLite foreign keys
- SQLite does not enforce foreign keys unless PRAGMA foreign_keys is enabled per-connection, so the connector activates it on each open to ensure relational integrity for users, tickets, and comments.[4]
- Enabling or disabling foreign_keys is a no-op within a transaction, so the connector sets it immediately after opening a connection before transactional work begins.[4]

